# 1. Main code.

In [1]:
from tqdm.auto import tqdm

In [2]:
%load_ext Cython

In [3]:
%%cython --annotate

from libc.math cimport log2
import numpy as np
cimport numpy as np
np.import_array()
import functools

#@functools.lru_cache(maxsize=1024)
cpdef ProcessRT(t, int i, int l, int nn1):
    """ This function recursively extends an input tuple of integers, which represents 
    a set of sets (more precisily, a formal context), by adding to it an integer i (a new set).
    The length of a terminal tuple is constrained by l, while nn1 should be passed as 2**n-1. 
    The function checks whether the new tuple forms a unique closure system and returns the dictionary of the first elements of
    processed tuples as keys and values 1 for clousre system with T1 propery and 0 otherwise."""
    
    count={}
    t = list(t)
    cdef int P
    cdef int lent = len(t)
    cdef int tji
    #cdef int tjh

    
    #checking whether the context is reducible
    cdef Py_ssize_t j, k, h
    for j in range(lent-1,-1,-1):
        P = i
        #print("t[j],i",t[j],i,t[j]&i,)
        tji = t[j] & i
        if tji in t[0:j]:
            return count
            
        if tji == t[j]:
            #print("t[j],i",t[j],i,t[j]&i,)
            for h in range(j + 1, lent):
                #tjh = 
                if t[j] & t[h] == t[j]:
                    P = P & t[h]
                    if P == t[j]:                        
                        return count
    t.append(i)
    tt = tuple(t)
    #checking T1 property
    if Check(tt, nn1):
        count[tt]=1
    else:
        count[tt]=0

    #recursive call
    #print(l)
    if l != 0:
        for k in range(i + 1, nn1):
            #print("Process(t,k,l-1,tuples,nn1)")
            #print("t",t,"k",k)
            ucount=ProcessRT(tt, k, l-1, nn1)
            for key in ucount:
                if key in count:
                    count[key] += ucount[key]
                else:
                    count[key] = ucount[key]
            #if r<0: print("r",r)
        return count
    else: 
        #print(tuples)
        #tuples.append(tuple(t))
       
        
        print("Branch process", t[0])
        return count

def isClosed(t, int g):
    """Checks whether g is closed in t."""
    cdef unsigned int h
    cdef list cont = [h for h in t if g&h == g]
    #for h in t:
    #    if g&h == g:
    #        cont.append(h)
    if len(cont) > 0:
        res = cont[0]
        for h in cont:
            res = h&res
            
        #res = np.bitwise_and.reduce(cont) & cont[0]
        return res == g
    else:
        return g == 0

def Check(t,int nn1):
    """Checks whether T1 is fulfilled for t."""
    cdef int n = int(log2(nn1+1))
    cdef int i
    cdef int m
    cdef list members = []
    members.extend([2**i for i in range(n)])
    for m in members:
        if not isClosed(t, m):
            return False
    return True

cpdef merge(r):
    """This is a value collector for the resulting list of dictionaries
    """
    merged = {}
    for dic in r:
        for key in dic:
            if key in merged:
                merged[key]+=dic[key]
            else:
                merged[key]=dic[key]
    return merged


def merge_nz(r):
    """This is a value collector for the resulting list of dictionaries
    with non-zero values.
    """
    merged={}
    for dic in r:
        for key in dic:
            if key in merged and dic[key]==1:
                merged[key]+=dic[key]
            else:
                if dic[key]==1:
                    merged[key]=dic[key]
    return merged

cpdef PermBit(int bit, perm):
    cdef int out = 0
    cdef int i
    cdef Py_ssize_t lp = len(perm)
    for i in range(lp):
        if (bit>>i) & 1==1:
            out = out | (1 << perm[i])
    return out

def PermTuple(t, perm):
    cdef int bit
    return tuple(PermBit(bit, perm) for bit in t)


cpdef isLectLess(a,b):
    cdef int i
    cdef Py_ssize_t l = len(a)
    for i in range(l):
        if a[i]==b[i]:
            pass
        elif a[i]<b[i]:
            return True
        else:
            return False
    return True
        
        

# pure py

In [10]:

#from libc.math cimport log2
from math import log
import copy
import functools
#@functools.lru_cache(maxsize=1024)
def ProcessRT(t, i, l, nn1):
    """ This function recursively extends an input tuple of integers, which represents 
    a set of sets (more precisily, a formal context), by adding to it an integer i (a new set).
    The length of a terminal tuple is constrained by l, while nn1 should be passed as 2**n-1. 
    The function checks whether the new tuple forms a unique closure system and returns the dictionary of the first elements of
    processed tuples as keys and values 1 for clousre system with T1 propery and 0 otherwise."""
    
    count={}
    t = copy.copy(t)
    lent=len(t)

    
    #checking whether the context is reducible
    for j in range(lent-1,-1,-1):
        P = i
        #print("t[j],i",t[j],i,t[j]&i,)
        tji = t[j] & i
        if tji in t[0:j]:
            return count
            
        if tji == t[j]:
            #print("t[j],i",t[j],i,t[j]&i,)
            for h in range(j + 1, lent):
                if t[j] & t[h] == t[j]:
                    P = P & t[h]
                    if P == t[j]:                        
                        return count
    t.append(i)

    #checking T1 property
    if Check(t, nn1):
        count[(lent + 1, tuple(t))]=1
    else:
        count[(lent + 1, tuple(t))]=0

    #recursive call
    #print(l)
    if l != 0:
        r = 0
        for k in range(i + 1, nn1):
            #print("Process(t,k,l-1,tuples,nn1)")
            #print("t",t,"k",k)
            ucount=ProcessRT(t, k, l-1, nn1)
            for key in ucount:
                if key in count:
                    count[key] += ucount[key]
                else:
                    count[key] = ucount[key]
            #if r<0: print("r",r)
        return count
    else: 
        #print(tuples)
        #tuples.append(tuple(t))
       
        
        print("Branch process", t[0])
        return count

    
def isClosed(t, g):
    """Checks whether g is closed in t."""
    cont=[]
    for h in t:
        if g&h == g:
            cont.append(h)
    if len(cont) > 0:
        res = cont[0]
        for h in cont:
            res = h&res
        return res == g
    else:
        return g == 0

    
def Check(t,nn1):
    """Checks whether T1 is fulfilled for t."""
    n = int(log(nn1+1, 2))
    members=[]
    members.extend([2**i for i in range(n)])
    for m in members:
        if not isClosed(t,m):
            return False
    return True

def merge(r):
    """This is a value collector for the resulting list of dictionaries
    """
    merged={}
    for dic in r:
        for key in dic:
            if key in merged:
                merged[key]+=dic[key]
            else:
                merged[key]=dic[key]
    return merged


def merge_nz(r):
    """This is a value collector for the resulting list of dictionaries
    with non-zero values.
    """
    merged={}
    for dic in r:
        for key in dic:
            if key in merged and dic[key]==1:
                merged[key]+=dic[key]
            else:
                if dic[key]==1:
                    merged[key]=dic[key]
    return merged

def PermBit(bit, perm):
    out=0
    for i in range(len(perm)):
        if (bit>>i) & 1==1:
            out=out | (1<<perm[i])
    return out

def PermTuple(t,perm):
    ret = [PermBit(bit,perm) for bit in t]
    return tuple(ret)


def isLectLess(a,b):
    for i in range(len(a)):
        if a[i]==b[i]:
            pass
        elif a[i]<b[i]:
            return True
        else:
            return False
    return True
        
        

# 2. Counting closure systems from A334254 for n=2, 3, 4, 5.

In [ ]:
from itertools import permutations
from cython.parallel import prange
from time import time
n = 6

if __name__ == "__main__":
    
    start = time()
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}


    for t in range(nn1):

        for i in range(t+1,nn1):
            lt.append(tuple([t]))
            lk.append(i)

    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)

    args = list(zip(lt,lk,ln_2,lnn1))
    print(args)
    
    res = [ProcessRT(*args[i]) for i in tqdm(prange(len(args), num_threads=8, nogil=True))]
    end_main = time() - start
    merged_res = merge(res)
    merged_nz_res = merge_nz(res)
    print(sum(merged_res.values()))
    
    T = list(merged_nz_res.keys())
    ineq_res = []
    
    Perms = list(permutations(range(n)))
    for t in T:
        mode=True
        for p in Perms:
            pt = sorted(PermTuple(t,p))
            if isLectLess(t,pt):
                pass
            else:
                mode=False
                break

        if mode: 
            ineq_res.append(t)
    end_ineq = (time() - start) - end_main
    print(len(ineq_res))
    print(f'Main: {end_main:.1f}s, ineq: {end_ineq:.1f}s')

[((0,), 1, 62, 63), ((0,), 2, 62, 63), ((0,), 3, 62, 63), ((0,), 4, 62, 63), ((0,), 5, 62, 63), ((0,), 6, 62, 63), ((0,), 7, 62, 63), ((0,), 8, 62, 63), ((0,), 9, 62, 63), ((0,), 10, 62, 63), ((0,), 11, 62, 63), ((0,), 12, 62, 63), ((0,), 13, 62, 63), ((0,), 14, 62, 63), ((0,), 15, 62, 63), ((0,), 16, 62, 63), ((0,), 17, 62, 63), ((0,), 18, 62, 63), ((0,), 19, 62, 63), ((0,), 20, 62, 63), ((0,), 21, 62, 63), ((0,), 22, 62, 63), ((0,), 23, 62, 63), ((0,), 24, 62, 63), ((0,), 25, 62, 63), ((0,), 26, 62, 63), ((0,), 27, 62, 63), ((0,), 28, 62, 63), ((0,), 29, 62, 63), ((0,), 30, 62, 63), ((0,), 31, 62, 63), ((0,), 32, 62, 63), ((0,), 33, 62, 63), ((0,), 34, 62, 63), ((0,), 35, 62, 63), ((0,), 36, 62, 63), ((0,), 37, 62, 63), ((0,), 38, 62, 63), ((0,), 39, 62, 63), ((0,), 40, 62, 63), ((0,), 41, 62, 63), ((0,), 42, 62, 63), ((0,), 43, 62, 63), ((0,), 44, 62, 63), ((0,), 45, 62, 63), ((0,), 46, 62, 63), ((0,), 47, 62, 63), ((0,), 48, 62, 63), ((0,), 49, 62, 63), ((0,), 50, 62, 63), ((0,), 5

  0%|          | 0/1953 [00:00<?, ?it/s]